In [ ]:
from langchain.llms import OpenAI


In [11]:
import os
os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"

In [ ]:
os.environ["GEMINI_API_KEY"] = "your_gemini_api_key_here"

In [14]:
llm=OpenAI(openai_api_key=os.environ["OPENAI_API_KEY"],temperature=0.6)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.7)

response = llm.invoke("Explain quantum computing in simple terms.")
print(response)


In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "your_huggingface_token_here"

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint

# Instantiate the model
llm_huggingface_endpoint = HuggingFaceEndpoint(
    repo_id="google/flan-t5-base",
    temperature=0.1,
    max_new_tokens=64  # ✅ correct keyword for this model
)

# Use invoke (not direct call)
response = llm_huggingface_endpoint.invoke("What is the capital of France?")
print(response)


### Prompt Template and LLM Chain


In [57]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    input_variables=["country"], template="What is the capital of {country}?")
print(prompt_template.format(country="France"))

What is the capital of France?


In [ ]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt_template)
print(chain.invoke("France"))

### Combining multiple chains using Simple Sequential Chain

In [ ]:
capital_template=PromptTemplate(input_variables=['country'],
template="Please tell me the capital of the {country}")

capital_chain=LLMChain(llm=llm,prompt=capital_template)

famous_template = PromptTemplate(input_variables=['capital'],
template="Suggest me some amazing places to visit in {capital}")

In [ ]:
famous_chain = LLMChain(llm=llm, prompt = famous_template)

In [ ]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains = [capital_chain, famous_chain]) ## capital_chain output becomes the input to famous_chain
chain.invoke("India")

# Output:
# " It is a bustling metropolis and a great place to visit for its historical sites, cultural attractions, and modern attractions. Here are some of the amazing places to visit in New Delhi: \n\n1. Red Fort: This 17th century Mughal fort is a UNESCO World Heritage Site and is one of the most popular tourist attractions in the city. \n\n2. India Gate: This iconic war memorial and national monument is a must-visit. It stands as a symbol of the sacrifice of the Indian soldiers who fought in World War I.\n\n3. Humayun's Tomb: This 16th century Mughal-era tomb is a UNESCO World Heritage Site and is one of the most important monuments in Delhi.\n\n4. Qutub Minar: This 73-meter-high tower is a UNESCO World Heritage Site and is one of the most iconic structures in Delhi.\n\n5. Jama Masjid: This 17th century mosque is one of the largest and most beautiful mosques in India.\n\n6. Lotus Temple: This modern temple is a Bahá'í House of Worship and is a popular tourist attraction.\n\n7. Akshardham Temple: This modern Hindu temple complex is a popular tourist destination"

## Sequential Chain

In [69]:
capital_template=PromptTemplate(input_variables=['country'],
template="Please tell me the capital of the {country}")

capital_chain=LLMChain(llm=llm,prompt=capital_template,output_key="capital")

In [70]:
famous_template=PromptTemplate(input_variables=['capital'],
template="Suggest me some amazing places to visit in {capital}")

famous_chain=LLMChain(llm=llm,prompt=famous_template,output_key="places")

In [ ]:
from langchain.chains import SequentialChain

chain=SequentialChain(chains=[capital_chain,famous_chain],
input_variables=['country'],
output_variables=['capital',"places"])

In [ ]:
chain.invoke({'country':"India"})

# Output:
# {'country': 'India',
#  'capital': '\n\nThe capital of India is New Delhi.',
#  'places': ' Here are some amazing places to visit in New Delhi: \n\n1. Red Fort: The majestic Red Fort is a 17th-century fort complex built by Mughal Emperor Shah Jahan. It is a UNESCO World Heritage Site and is a must-visit for all tourists. \n\n2. India Gate: India Gate is a 42 meter-high sandstone archway built by Edwin Lutyens in 1931. It is a memorial to the Indian soldiers who lost their lives during World War I. \n\n3. Qutub Minar: The Qutub Minar is a 73 meter-high tower built by Qutb-ud-din Aibak in 1193. It is a UNESCO World Heritage Site and is the tallest brick minaret in the world. \n\n4. Humayun’s Tomb: Humayun’s Tomb is a 16th-century tomb built by Mughal Emperor Humayun. It is a UNESCO World Heritage Site and is a great example of Mughal architecture. \n\n5. Jama Masjid: The Jama Masjid is a 17th-century mosque built by Mughal Emperor Shah Jahan. It is one of the largest'}

## Chatmodels With ChatOpenAI

In [1]:
from langchain.chat_models import ChatOpenAI

In [2]:
from langchain.schema import HumanMessage,SystemMessage,AIMessage

In [ ]:
import os
chatllm = ChatOpenAI(openai_api_key = os.environ["OPENAI_API_KEY"], temperature = 0.6, model = 'gpt-3.5-turbo')

In [ ]:
chatllm([
    SystemMessage(content="Yor are a comedian AI assitant"),
    HumanMessage(content="Please provide some comedy punchlines on AI")
])

## Output:
# AIMessage(content='1. "AI may be smart, but can it tell me if my outfit makes me look like a potato?"\n2. "AI is like a virtual therapist, except it never judges you for eating an entire pizza by yourself."\n3. "AI is great at predicting the future, but can it predict when my pizza delivery will actually arrive?"\n4. "They say AI can learn from its mistakes, but I\'m still waiting for it to apologize for recommending me that terrible movie."\n5. "AI may be able to beat humans at chess, but can it figure out how to untangle a pair of earphones?"\n6. "AI is like a high-tech fortune teller, except it tells you what you\'re going to have for dinner instead of your future."\n7. "AI is so advanced, it can even make my phone autocorrect my perfectly spelled words into complete nonsense."\n8. "AI may be able to recognize faces, but can it recognize when someone\'s had a bad haircut?"\n9. "AI is like having a personal assistant, except it never judges you for spending hours watching cat videos on YouTube."\n10. "AI is great at analyzing data, but can it analyze why I can never find matching socks in my drawer?"')

## Prompt Template + LLM +Output Parsers

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [7]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")

In [8]:
template="Your are a helpful assistant. When the use given any input , you should generate 5 words synonyms in a comma seperated list"
human_template="{text}"
chatprompt=ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)
])

In [ ]:
chain=chatprompt|chatllm|Commaseperatedoutput()

In [ ]:
chain.invoke({"text":"intelligent"})
##Output:
# ['smart', 'clever', 'bright', 'brainy', 'shrewd']